In [1]:
import pandas as pd


In [116]:
df_distances = pd.read_csv("../../results_demo/T50/FFT/Group_1/Hamming/distance_estimator.csv")
df_distances

,id_seq1,target_seq1,id_seq2,target_seq2,distance
0,sequence_53082,55.0,sequence_19402,43.0,1.0
1,sequence_53082,55.0,sequence_20054,49.0,1.0
2,sequence_53082,55.0,sequence_20048,39.8,1.0
3,sequence_53082,55.0,sequence_20036,52.9,1.0
4,sequence_53082,55.0,sequence_19525,48.8,1.0
...,...,...,...,...,...
67855,sequence_19401,47.4,sequence_53056,52.0,1.0
67856,sequence_19401,47.4,sequence_19347,58.9,1.0
67857,sequence_19401,47.4,sequence_19379,50.3,1.0
67858,sequence_19401,47.4,sequence_19348,46.7,1.0


In [117]:
#filtramos las distancias para trabajar con solo una parte de ellas -> nos interesan solo las más cercanas
import numpy as np

q1 = np.quantile(df_distances['distance'], .25)
q3 = np.quantile(df_distances['distance'], .75)

print(q3, q1)
IQR = q3 - q1
IQR = 1 * IQR

low_freq = q1 - IQR
high_freq = q3 + IQR

print(low_freq, high_freq)

df_filter = df_distances.loc[df_distances['distance'] <= q3]
df_filter = df_filter.reset_index()

1.0 1.0
1.0 1.0


In [118]:
df_filter

,index,id_seq1,target_seq1,id_seq2,target_seq2,distance
0,0,sequence_53082,55.0,sequence_19402,43.0,1.0
1,1,sequence_53082,55.0,sequence_20054,49.0,1.0
2,2,sequence_53082,55.0,sequence_20048,39.8,1.0
3,3,sequence_53082,55.0,sequence_20036,52.9,1.0
4,4,sequence_53082,55.0,sequence_19525,48.8,1.0
...,...,...,...,...,...,...
67855,67855,sequence_19401,47.4,sequence_53056,52.0,1.0
67856,67856,sequence_19401,47.4,sequence_19347,58.9,1.0
67857,67857,sequence_19401,47.4,sequence_19379,50.3,1.0
67858,67858,sequence_19401,47.4,sequence_19348,46.7,1.0


In [119]:
import community as community_louvain
import networkx as nx

In [120]:
graph_data = nx.Graph()

id_list1 = df_filter['id_seq1'].to_list()
id_list2 = df_filter['id_seq2'].to_list()

nodos = id_list1 + id_list2
nodos = list(set(nodos))

#cargar nodos
for node in nodos:
    graph_data.add_node(node)

#cargar aristas
for i in range(len(df_filter)):
    graph_data.add_edge(df_filter['id_seq1'][i],
        df_filter['id_seq2'][i],
        weigth=df_filter['distance'][i]
    )

In [121]:
partition = community_louvain.best_partition(graph_data)
modularity_value = round(community_louvain.modularity(partition, graph_data), 3)

print("Modularity: ", modularity_value)

Modularity:  0.0


In [122]:
#obtenemos los grupos
matrix_group = []
for element in partition:
    row = [element, partition[element]]
    matrix_group.append(row)
results = pd.DataFrame(matrix_group, columns=['id', 'label'])

results.to_csv("../../results_demo/T50/FFT/Group_1/Hamming/graph_clustering_sequences.csv", index=False)